In [1]:
!pip install -U -q PyDrive

In [2]:
import sqlite3
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
!mkdir data

In [6]:
file_id = '1zQ8AR2Ry3ajzB3UZO1Sfk3xtDJlzQF2M'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/checking-logs.sqlite')

In [7]:
conn = sqlite3.connect('data/checking-logs.sqlite')

In [8]:
datamart = pd.io.sql.read_sql(
    """
    select checker.uid, checker.labname, checker.timestamp as checker_ts, pageviews.datetime as first_view_ts
    from checker left join pageviews on checker.uid = pageviews.uid
    where checker.status='ready' and checker.numTrials=1 and checker.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') and checker.uid like 'user_%' and
        (
            pageviews.datetime is null or pageviews.datetime =
                (
                    select min(pageviews.datetime)
                    from pageviews
                    where uid = checker.uid
                )
        )
    """,
    conn, parse_dates=['checker_ts', 'first_view_ts']
    )

In [9]:
datamart.head(10)

,uid,labname,checker_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
9,user_12,laba04,2020-04-18 17:07:51.767358,NaT


In [10]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uid            140 non-null    object        
 1   labname        140 non-null    object        
 2   checker_ts     140 non-null    datetime64[ns]
 3   first_view_ts  59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [11]:
test = datamart.loc[datamart['first_view_ts'].notnull()]
test = test.reset_index(drop=True)

In [12]:
test.head()

,uid,labname,checker_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uid            59 non-null     object        
 1   labname        59 non-null     object        
 2   checker_ts     59 non-null     datetime64[ns]
 3   first_view_ts  59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.0+ KB


In [15]:
control = datamart.loc[datamart['first_view_ts'].isnull()]
control = control.reset_index(drop=True)

In [17]:
control = control.fillna(test['first_view_ts'].mean())

In [18]:
control.head()

,uid,labname,checker_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783808
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783808
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783808
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783808
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783808


In [20]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uid            81 non-null     object        
 1   labname        81 non-null     object        
 2   checker_ts     81 non-null     datetime64[ns]
 3   first_view_ts  81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB


In [21]:
test.to_sql('test', conn)

In [22]:
control.to_sql('control', conn)

In [23]:
conn.close()

In [24]:
uploaded = drive.CreateFile({'title': 'checking-logs.sqlite'})
uploaded.SetContentFile('data/checking-logs.sqlite')
uploaded.Upload()